In [1]:
%matplotlib qt5
import numpy as np
from numpy import linalg as la
import matplotlib.pylab as plt
from scipy.optimize import fsolve
import math
import statsmodels.api as sm
from scipy.stats import norm
import seaborn as sns
from scipy import stats
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

In [2]:
import scipy
from functools import partial
def odeIntegral(x,t,J,I=0):
    x = np.squeeze(x)
    x = np.reshape(x,(len(x),1))
    # dxdt = -x+J@np.tanh(x)#+I[0]
    dxdt = -x+J@x+I[0]
    return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)

In [58]:
''' Parameters used for all networks (reciprocal and chain)'''
### get the current path location and read the data file
import os
strr= os.getcwd()
print(strr)

### define the network parameters of the adjacency matrix
N  = 1500
Kt = int(N*0.2)
J = 1/np.sqrt(N)*0.5  ### TODO: make sure this scalar with David&Stefano's paper
J = 0.00325
ntau   = 10
trials = 30+6
tau_series = np.linspace(0,0.15,ntau)#
### EEII; EEEIII; EEEIII;EEII
switch = 1
## arrays to store results
## norml0_series: norm of left eigenvector(deltaliri = 1)
eigvchn_series, eigrvec_series, eiglvec_series = np.zeros((trials,ntau,N),dtype=complex), np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
eiglvec0_series, norml0_series = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau+1,N)) 
### also have the reconstructed left and right eigenvectors 
eigrvec_series_rec, eiglvec_series_rec = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
htau = tau_series[1]-tau_series[0]
### simulation using the low-rank framework
firing_rateeq = np.zeros((trials,ntau,N))
### recording dynamics
lowrank_eq, lowrank_eq_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,N))
ovs_inplr, ovs_inplr_num   = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
ovs_inplr_div, ovs_inplr_div_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
contributions_lr, contributions_lr_num = np.zeros((trials,ntau,2,2)), np.zeros((trials,ntau,2,2))### rank, population

## numerical 
intg_ov_series    = np.zeros((trials,ntau,2))
first_perturb_ov  = np.zeros((trials,ntau,2,2))
first_perturb_ovP = np.zeros((trials,ntau,2,2,2)) ### rank and population

intg_mean_series  = np.zeros((trials,ntau,N,2,2))### rank2 and population2
intg_std_series   = np.zeros((trials,ntau,2,2))### rank, population
intg_std_num_series = np.zeros((trials,ntau,2,2))### rank, population
mean_shift_ov     = np.zeros((trials,ntau,2))
latent_kappa_series = np.zeros((trials,ntau,2,2))
intg_crossov_series = np.zeros((trials,ntau,2,2)) # rank, rank
intg_crossovPop_series = np.zeros((trials,ntau,2,2,2)) # rank, rank, population
g, gamma =  6, 1/4.0
NE = int(N/(1+gamma))
NI = N-NE#NE*gamma
N  = NE+NI ### update 
ALPHAE, ALPHAI = NE/N, NI/N
KE, KI = int(Kt/(1+gamma)), int(Kt/(1+gamma)*gamma) ### fixed out-degree
ce, ci = KE/NE, KI/NI
print('ce and ci:',ce,ci)
### assert that the differences between ce and ci are smaller than epsilon
epsilon = 1E-2
assert np.abs(ce-ci)<epsilon
# assert ce==ci
c = ce
ji,je = g*J,J 
### define the network parameters of the diluted Gaussian matrix 
ge, gi = np.sqrt(je**2*c*(1-c)*N), np.sqrt(ji**2*c*(1-c)*N) 
hat_sigmae, hat_sigmai = np.sqrt(c*(1-c)), np.sqrt(c*(1-c))### standard deviation of the adjacency matrix
sigmae,sigmai = np.sqrt(c*(1-c)*J**2*N), np.sqrt(c*(1-c)*(-g*J)**2*N)### with magnitude of the coupling
JE,JI = je*c*NE, ji*c*NI 
lambda0 = JE-JI 
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 
## TEST THE EIGENVALUES OF THE MEAN MATRIX 
eigvJ0, eigvecJ0 = la.eig(Jbar)
print('eigvJ0:',eigvJ0[0],' theory:',JE-JI)
### mean left and right eigenvectors
leigvec0, reigvec0 = np.zeros((N,N)), np.zeros((N,N))
norm_left = np.zeros(2)
## first eigenvector
leigvec0[:,0], reigvec0[:,0] = nvec[:,0]/(JE-JI)/np.sqrt(N), mvec[:,0]/np.sqrt(N)
norm_left[0]  = la.norm(leigvec0[:,0])
leigvec0[:,0] = leigvec0[:,0]/norm_left[0]
norml0_series[:,0,0] = np.sum(leigvec0[:,0]*reigvec0[:,0])
## second eigenvector
kk = np.sqrt(NE*JI**2+NI*JE**2)
reigvec0[:NE,1], reigvec0[NE:,1] = JI/kk,JE/kk 
leigvec0[:NE,1], leigvec0[NE:,1] = -kk/(JE-JI)/NE,kk/(JE-JI)/NI 
norm_left[1]     = la.norm(leigvec0[:,1])
leigvec0[:,1] = leigvec0[:,1]/norm_left[1]
norml0_series[:,0,1] = np.sum(leigvec0[:,1]*reigvec0[:,1])
outerproduct  = np.sum(leigvec0[:,0]*reigvec0[:,1])#*norm_left
print('JE:',JE,'JI:',JI)


e:\Dropbox\DailyWork\Allen_project\Preparation_Paper_Figures\PRX_prune\PRX_numeric
ce and ci: 0.2 0.2
eigvJ0: (-0.3899999999999837+0j)  theory: -0.39000000000000024
JE: 0.7799999999999999 JI: 1.1700000000000002


In [59]:
# for value in {0.   0.14 0.28 0.42 0.56 0.7  0.84 0.98 1.12 1.26 1.4 }; do
#     ./run_secorder 1000 0.2 $(echo "$value*2" | bc) $value $value 1 1
# done
# for seed in {1 7 8 12 13 19 28 31 33 39 40 42 48 50 51 52 55 56 59 65 66 67 69 70 75 76 78 82 83 86 94}; do     mkdir -p "seed_$seed"; done

In [60]:
# ntau = 10#21#
# trials = 6#30+6
# tau_series = np.linspace(0,0.15,ntau)#
# alpha_series = (tau_series*c*(1-c)+c**2)/c**2-1
# alpha_series_rec = (tau_series*2*c*(1-c)+c**2)/c**2-1
# print(alpha_series)
# print(alpha_series_rec)
# ((alpha_series+1)*c**2-c**2)/c/(1-c)

In [61]:
# tau_series = np.array([0.025, 0.05 , 0.075, 0.1  ])
eigvchn_series, eigrvec_series, eiglvec_series = np.zeros((trials,ntau,N),dtype=complex), np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
eiglvec0_series, norml0_series = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau+1,N)) 
### normalization for left and right eigenvectors 
norms_for_lvec, norms_for_rvec = np.zeros((trials,ntau+1,2)), np.zeros((trials,ntau+1,2))
### also have the reconstructed left and right eigenvectors 
eigrvec_series_rec, eiglvec_series_rec = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 
## TEST THE EIGENVALUES OF THE MEAN MATRIX 
eigvJ0, eigvecJ0 = la.eig(Jbar)
print('eigvJ0:',eigvJ0[0],' theory:',JE-JI)
### mean left and right eigenvectors
leigvec0, reigvec0 = np.zeros((N,N)), np.zeros((N,N))
norm_left = np.zeros(2)
## first eigenvector
leigvec0[:,0], reigvec0[:,0] = nvec[:,0]/(JE-JI)/np.sqrt(N), mvec[:,0]/np.sqrt(N)
norm_left[0]  = la.norm(leigvec0[:,0])
leigvec0[:,0] = leigvec0[:,0]/norm_left[0]
norml0_series[:,0,0] = np.sum(leigvec0[:,0]*reigvec0[:,0])
## second eigenvector
kk = np.sqrt(NE*JI**2+NI*JE**2)
reigvec0[:NE,1], reigvec0[NE:,1] = JI/kk,JE/kk 
leigvec0[:NE,1], leigvec0[NE:,1] = -kk/(JE-JI)/NE,kk/(JE-JI)/NI 
norm_left[1]  = la.norm(leigvec0[:,1])
leigvec0[:,1] = leigvec0[:,1]/norm_left[1]
norml0_series[:,0,1] = np.sum(leigvec0[:,1]*reigvec0[:,1])
outerproduct = np.sum(leigvec0[:,0]*reigvec0[:,1])#*norm_left
### compute the square of the random connectivity 
Z2E = N*J**2*hat_sigmae**2*tau_series*ALPHAE-N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAI
Z2I = -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAE+N*g**2*J**2*hat_sigmai**2*tau_series*ALPHAI
DeltaZ2E, DeltaZ2I = Z2E[1]-Z2E[0],Z2I[1]-Z2I[0] ### for practical use
print('DeltaZ2E,DeltaZ2I:',DeltaZ2E,DeltaZ2I)
DeltaZ2 = np.zeros((N,N))
DeltaZ2[:,:NE], DeltaZ2[:,NE:] = DeltaZ2E, DeltaZ2I

### E population and I population separately
Z2E_E, Z2I_E = N*J**2*hat_sigmae**2*tau_series*ALPHAE, -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAI
Z2E_I, Z2I_I = -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAE, N*g**2*J**2*hat_sigmai**2*tau_series*ALPHAI
DeltaZ2E_E, DeltaZ2I_E = Z2E_E[1]-Z2E_E[0],Z2I_E[1]-Z2I_E[0] ### for practical use
DeltaZ2E_I, DeltaZ2I_I = Z2E_I[1]-Z2E_I[0],Z2I_I[1]-Z2I_I[0] ### for practical use
DeltaZ2_E, DeltaZ2_I = np.zeros((N,N)),np.zeros((N,N))
DeltaZ2_E[:,:NE], DeltaZ2_E[:,NE:] = DeltaZ2E_E, DeltaZ2I_E

eigvJ0: (-0.3899999999999837+0j)  theory: -0.39000000000000024
DeltaZ2E,DeltaZ2I: -1.690000000000001e-05 0.00010140000000000003


### Chain motifs

In [62]:
# alphachn_series = np.array([0.,0.07,0.13,0.2,0.27,0.33,0.4,0.47,0.53,0.6])


In [63]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
Ipert = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
# Ipert[NE:]=0
Ipert[:NE]=0
tt = np.linspace(0,100,1000)
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

In [64]:
ce=ci=c
### generate i.i.d. s
def randbin(M,N,P):  
    return np.random.choice([0, 1], size=(M,N), p=[P, 1-P])

sparseiid_dyns = np.zeros((trials,N,1))
for ktrial in range(trials):
    ### generate iid sparse connectivity 
    Ecol = randbin(1,NE*N,1-ce)
    Icol = randbin(1,NI*N,1-ci)
    Ecol = np.reshape(Ecol,(N,NE))
    Icol = np.reshape(Icol,(N,NI))
    iidsparse = np.zeros((N,N))
    iidsparse[:,:NE], iidsparse[:,NE:] = Ecol.copy()*J,Icol.copy()*J*(-g)
    xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
    xtemporal = odesimulation(tt, xinit, iidsparse, Inp)
    firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))
    sparseiid_dyns[ktrial,:,:] = firing_rate0.copy()
    
sparseiid_dyns = np.squeeze(sparseiid_dyns)
mean_sparseiid_dyns = np.zeros((trials,2))
for ktrial in range(trials):
    mean_sparseiid_dyns[ktrial,0],mean_sparseiid_dyns[ktrial,1]= np.mean(sparseiid_dyns[ktrial,:NE]),np.mean(sparseiid_dyns[ktrial,NE:])
trialavg_sparseiid_dyns = np.mean(mean_sparseiid_dyns,axis=0)
trialstd_sparseiid_dyns = np.std(mean_sparseiid_dyns,axis=0)
print(trialavg_sparseiid_dyns)

[0.01853041 0.01857323]


In [65]:
### simulation using the low-rank framework
firing_rateeq = np.zeros((trials,ntau,N))
### recording dynamics
lowrank_eq, lowrank_eq_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,N))
ovs_inplr, ovs_inplr_num   = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
ovs_inplr_div, ovs_inplr_div_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
contributions_lr, contributions_lr_num = np.zeros((trials,ntau,2,2)), np.zeros((trials,ntau,2,2))### rank, population


In [66]:
'_1200_300_0.200_0.200_0.200_0.200_1.20_0.00_1.20_0.60_0.00_0.00_0.60_0.00_0.00_0.60_0.60_0.60_0.60_1.00_1.00_1.00_0.00_0.00_1.00_1.00_0.00_0.00_1.00'
alphachn_series = np.array([0.,0.07,0.13,0.2,0.27,0.33,0.4,0.47,0.53,0.6])
alphachns     = ['0.00','0.07','0.13','0.20','0.27','0.33','0.40','0.47','0.53','0.60']
alphachns_rec = ['0.00','0.14','0.26','0.40','0.54','0.66','0.80','0.94','1.06','1.20']
# alphachns_arr = np.array([0.1,0.2,0.3,0.4])
eigvchn_series = np.zeros((trials,len(alphachns),N),dtype=complex)

firing_ratepert = np.zeros((trials,ntau,N))
# tauchns_arr = alphachns_arr*c**2/c/(1-c)
seedss = np.array([1,2,3,4,5,6])
seedss = np.arange(1,38)

for ktrial in range(trials):
        seed =seedss[ktrial]
        z_pre = 0
        for ialpha in range(len(alphachns)):
            file_path = 'K:\\Projects\\Allen_motif\\PRXL_data\\sonets\\2populations\\it'+str(ialpha)+'\\w_'+str(seed)+'_1200_300_0.200_0.200_0.200_0.200_'+alphachns_rec[ialpha]+'_0.00_'+alphachns_rec[ialpha]+'_'+alphachns[ialpha]+'_0.00_0.00_'+alphachns[ialpha]+'_0.00_0.00_'+alphachns[ialpha]+'_'+alphachns[ialpha]+'_'+alphachns[ialpha]+'_'+alphachns[ialpha]+'_1.00_1.00_1.00_0.00_0.00_1.00_1.00_0.00_0.00_1.00'+'.dat'
            Jmat  = np.zeros((N,N))
            count = 0
            with open(file_path, 'r') as file:
                content = file.read()
                if content == '0':
                    print('File is empty') 
                else:
                    for line in content.split('\n'):
                        if line:
                            Jmat[count,:]=([(x) for x in line.split()])
                            count +=1
                Jmat = np.zeros((N,N))
                count=0
                with open(file_path, 'r') as file:
                    content = file.read()
                    if content == '0':
                        print('File is empty') 
                    else:
                        for line in content.split('\n'):
                            if line:
                                Jmat[count,:]=([(x) for x in line.split()])
                                count +=1       
                Jmat[:,:NE] = Jmat[:,:NE]*je
                Jmat[:,NE:] = Jmat[:,NE:]*(-ji)

                ktrial, it = ktrial, ialpha
                ### full rank simulation
                xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
                xc_temporal = odesimulation(tt, xinit, Jmat, Inp)
                firing_rateeq[ktrial,it,:] = xc_temporal[-1,:].copy()
                
                ### perturbation 
                xpert = xc_temporal[-1,:].copy()
                xpert = xpert.reshape(-1,1)
                dtt =tt[1]-tt[0]
                for ttt in range(len(tt)):
                    delta_x= -xpert + Jmat@xpert.reshape(-1,1)+Ipert.reshape(-1,1)+Inp.reshape(-1,1)
                    xpert = delta_x*dtt+xpert 
                firing_ratepert[ktrial,it,:] = xpert.copy().squeeze()
                    
                zr = Jmat - Jbar
                ### compute the eigenvalues and eigenvectors of the matrix
                eigvchn, eigrvec = la.eig(Jmat)
                ## sort the eigenValues by it's norm of the real part 
                idxeigv = np.argsort(np.abs(eigvchn))
                idxeigv = idxeigv[::-1]
                eigvchn, eigrvec = eigvchn[idxeigv],eigrvec[:,idxeigv]
                
                eigvchn_,eiglvec = la.eig(Jmat.copy().T)
                idxeigv = np.argsort(np.abs(eigvchn_))
                idxeigv = idxeigv[::-1]
                eigvchn_, eiglvec = eigvchn[idxeigv],eiglvec[:,idxeigv]
                
                ### normalization
                reig    = np.squeeze(eigrvec[:,:].copy())
                leig0   = np.squeeze(eiglvec[:,:].copy()) 
                normval = np.sum(reig.copy()*leig0.copy(),axis=0)
                
                norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
                normval = np.repeat(np.reshape(normval,(1,N)),N,axis=0)
                leig = leig0.copy()/normval.copy()### left eigenvector normalization to make <l|r>=1
                if np.mean(reig[:NE,0])<0:
                    reig[:,0]*=-1
                    leig[:,0]*=-1
                    leig0[:,0]*=-1
                if np.mean(reig[:NE,1])<0: ### the second rank-1 component is negative
                    reig[:,1]*=-1
                    leig[:,1]*=-1
                    leig0[:,1]*=-1
                
                print('eigenvalues',eigvchn[:2])
                
                DELTA_Z = np.zeros((N,N))
                DELTA_Z = zr-z_pre  
                hzr = DELTA_Z.copy()
                
                eigvchn_series[ktrial,it,:]    = eigvchn[:] 
                eigrvec_series[ktrial,it,:,:]  = reig[:,:2].copy()#eigvecw_norm.copy()#
                eiglvec_series[ktrial,it,:,:]  = leig[:,:2].copy()#eigvect_norm.copy()#
                eiglvec0_series[ktrial,it,:,:] = leig0[:,:2].copy()#eigvect_norm.copy()#

                    
                DeltaZ2 = hzr@hzr    ### used to correct            
                
                z_pre = zr.copy()    

C:\Users\Administrator\AppData\Local\Temp\ipykernel_23508\1842915965.py:76: ComplexWarning: Casting complex values to real discards the imaginary part
  norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23508\1842915965.py:95: ComplexWarning: Casting complex values to real discards the imaginary part
  eigrvec_series[ktrial,it,:,:]  = reig[:,:2].copy()#eigvecw_norm.copy()#
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23508\1842915965.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  eiglvec_series[ktrial,it,:,:]  = leig[:,:2].copy()#eigvect_norm.copy()#
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23508\1842915965.py:97: ComplexWarning: Casting complex values to real discards the imaginary part
  eiglvec0_series[ktrial,it,:,:] = leig0[:,:2].copy()#eigvect_norm.copy()#


eigenvalues [-0.39988448+0.j          0.10742143+0.10556097j]
eigenvalues [-0.67093286+0.j  0.24360325+0.j]
eigenvalues [-0.79861671+0.j  0.34659565+0.j]
eigenvalues [-0.91144284+0.j  0.44588553+0.j]
eigenvalues [-1.01755125+0.j  0.5349248 +0.j]
eigenvalues [-1.10362934+0.j  0.60355298+0.j]
eigenvalues [-1.19435988+0.j  0.67645569+0.j]
eigenvalues [-1.27085033+0.j  0.74921037+0.j]
eigenvalues [-1.34773456+0.j  0.79892975+0.j]
eigenvalues [-1.42195984+0.j  0.85911242+0.j]
eigenvalues [-0.39394587+0.j          0.11272576-0.09415844j]
eigenvalues [-0.70293977+0.j  0.21509729+0.j]
eigenvalues [-0.84057234+0.j  0.34669324+0.j]
eigenvalues [-0.96763275+0.j  0.45122439+0.j]
eigenvalues [-1.08223921+0.j  0.54586616+0.j]
eigenvalues [-1.174283  +0.j  0.61868506+0.j]
eigenvalues [-1.26498465+0.j  0.69037175+0.j]
eigenvalues [-1.34039896+0.j  0.75931511+0.j]
eigenvalues [-1.4235742 +0.j  0.81109872+0.j]
eigenvalues [-1.50788877+0.j  0.87024875+0.j]
eigenvalues [-0.39855589+0.j         0.14763414+

C:\Users\Administrator\AppData\Local\Temp\ipykernel_23508\1842915965.py:78: RuntimeWarning: divide by zero encountered in divide
  leig = leig0.copy()/normval.copy()### left eigenvector normalization to make <l|r>=1
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23508\1842915965.py:78: RuntimeWarning: invalid value encountered in divide
  leig = leig0.copy()/normval.copy()### left eigenvector normalization to make <l|r>=1


eigenvalues [-1.4015987 +0.j  0.96349242+0.j]
eigenvalues [-0.33002941+0.j        -0.14429213+0.0546028j]
eigenvalues [-0.55871559+0.j         0.16783231+0.0286981j]
eigenvalues [-0.69151078+0.j  0.33431397+0.j]
eigenvalues [-0.80463977+0.j  0.44719461+0.j]
eigenvalues [-0.90356328+0.j  0.55095606+0.j]
eigenvalues [-0.98557091+0.j  0.6282354 +0.j]
eigenvalues [-1.0696952 +0.j  0.70767867+0.j]
eigenvalues [-1.1270489 +0.j  0.78579591+0.j]
eigenvalues [-1.19972551+0.j  0.83977567+0.j]
eigenvalues [-1.2589229 +0.j  0.90642564+0.j]
eigenvalues [-0.39418461+0.j         -0.07679915+0.12632971j]
eigenvalues [-0.64619063+0.j  0.23579647+0.j]
eigenvalues [-0.78631186+0.j  0.34347873+0.j]
eigenvalues [-0.91254015+0.j  0.45771496+0.j]
eigenvalues [-1.0134953 +0.j  0.55546962+0.j]
eigenvalues [-1.08210286+0.j  0.62892229+0.j]
eigenvalues [-1.1682997 +0.j  0.70566007+0.j]
eigenvalues [-1.22573758+0.j  0.78923371+0.j]
eigenvalues [-1.29930799+0.j  0.84087807+0.j]
eigenvalues [-1.3690497 +0.j  0.9092

In [67]:

# for seed in {0..36}; do
# ./run_secorder 1500 0.2 0.0 0.0 0.0 0 $seed
# ./run_secorder 1500 0.2 0.2 0.1 0.1 1 $seed
# ./run_secorder 1500 0.2 0.4 0.2 0.2 1 $seed
# ./run_secorder 1500 0.2 0.6 0.3 0.3 1 $seed
# ./run_secorder 1500 0.2 0.8 0.4 0.4 1 $seed
# ./run_secorder 1500 0.2 1.0 0.5 0.5 1 $seed
# ./run_secorder 1500 0.2 1.2 0.6 0.6 1 $seed
# ./run_secorder 1500 0.2 1.4 0.7 0.7 1 $seed
# ./run_secorder 1500 0.2 1.6 0.8 0.8 1 $seed
# ./run_secorder 1500 0.2 1.8 0.9 0.9 1 $seed

# done


# for seed in {1..37}; do
# ./run_secorder 2000 0.15 1.2 0.6 0.6 1 $seed
# done


# for seed in 1 4 8 10 13 14 18 19 25 26 31 32 34 36 40 43 44 49 52 53 55 60 62 64 71 73 76 78 81 82 83 84 85 86 87 88 93 94 95; do
# ./run_secorder 1500 0.2 0.0 0.0 0.0 0 $seed
# ./run_secorder 1500 0.2 0.2 0.1 0.1 1 $seed
# ./run_secorder 1500 0.2 0.4 0.2 0.2 1 $seed
# ./run_secorder 1500 0.2 0.6 0.3 0.3 1 $seed
# ./run_secorder 1500 0.2 0.8 0.4 0.4 1 $seed
# ./run_secorder 1500 0.2 1.0 0.5 0.5 1 $seed
# ./run_secorder 1500 0.2 1.2 0.6 0.6 1 $seed
# ./run_secorder 1500 0.2 1.4 0.7 0.7 1 $seed
# ./run_secorder 1500 0.2 1.6 0.8 0.8 1 $seed
# ./run_secorder 1500 0.2 1.8 0.9 0.9 1 $seed
# done

# for seed in 1 4 8 10 13 14 18 19 25 26 31 32 34 36 40 43 44 49 52 53 55 60 62 64 71 73 76 78 81 82 83 84 85 86 87 88 93 94 95; do     mkdir -p "seed_$seed"; done

In [68]:
''' DATA TO generate fig3'''
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1500_normlr_whilecode_used_vecrec.npz"
# sonets/equivalentGaussian/ConnStats_AdjGau_4April_PRX_68N1500_normlr_whilecode_used_vecrec.npz" #### equivalent Gaussian 

' DATA TO generate fig3'

In [69]:
def list_to_dict(lst, string):
    """
    Transform a list of variables into a dictionary.
    Parameters
    ----------
    lst : list
        list with all variables.
    string : str
        string containing the names, separated by commas.
    Returns
    -------
    d : dict
        dictionary with items in which the keys and the values are specified
        in string and lst values respectively.
    """
    string = string[0]
    string = string.replace(']', '')
    string = string.replace('[', '')
    string = string.replace('\\', '')
    string = string.replace(' ', '')
    string = string.replace('\t', '')
    string = string.replace('\n', '')
    string = string.split(',')
    d = {s: v for s, v in zip(string, lst)}
    return d

In [70]:
'''case 2'''
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = JE/NE, -JI/NI
def case1eigv(x,J,g,NE,NI,c,tau):
    tauc = np.abs(tau)
    zee = J**2*c*(1-c)*NE*tauc-g*J**2*c*(1-c)*NI*0
    zii = -g*J**2*c*(1-c)*NE*0+g**2*J**2*c*(1-c)*NI*tauc 
    # print((NE*zee+NI*zii)*4+(JE-JI)**2)
    z2lambda = np.zeros((N,N))
    z2lambda[:,:NE], z2lambda[:,NE:] = zee/x**2,zii/x**2 
    z2lambda = np.eye(N)-z2lambda 
    final = nvec.T@la.inv(z2lambda)@mvec 
    final = np.squeeze(final)
    return final-x

### solve x 
eigvchn_theo = np.zeros((ntau,2))
for it, tau in enumerate(tau_series):
    if it<=-1:
        continue
    x = fsolve(case1eigv,np.mean(eigvchn_series[:,it,1].real),args=(J,g,NE,NI,c,tau))
    eigvchn_theo[it,1] = x 
    print('tau:',tau,'eigv:',x)
    x = fsolve(case1eigv,np.mean((eigvchn_series[:,it,0]).real),args=(J,g,NE,NI,c,tau))
    eigvchn_theo[it,0] = x 
    
    print('tau:',tau,'eigv:',x)
    print('numerical:',np.mean(eigvchn_series[:,it,:2],axis=0))


tau: 0.0 eigv: [-0.39]
tau: 0.0 eigv: [-0.39]
numerical: [-0.36461304+0.00178754j  0.05109863+0.03914317j]
tau: 0.016666666666666666 eigv: [0.21712255]
tau: 0.016666666666666666 eigv: [-0.60712255]
numerical: [-0.65278072+0.j          0.23828578+0.00345895j]
tau: 0.03333333333333333 eigv: [0.35424038]
tau: 0.03333333333333333 eigv: [-0.74424038]
numerical: [-0.78887625+0.j  0.36499639+0.j]
tau: 0.05 eigv: [0.46339578]
tau: 0.05 eigv: [-0.85339578]
numerical: [-0.91757744+0.j  0.47807293+0.j]
tau: 0.06666666666666667 eigv: [0.55686767]
tau: 0.06666666666666667 eigv: [-0.94686767]
numerical: [-1.02846212+0.j  0.57394787+0.j]
tau: 0.08333333333333333 eigv: [0.63994012]
tau: 0.08333333333333333 eigv: [-1.02994012]
numerical: [-1.11351274+0.j  0.64823122+0.j]
tau: 0.1 eigv: [0.71546417]
tau: 0.1 eigv: [-1.10546417]
numerical: [-1.20529775+0.j  0.72697312+0.j]
tau: 0.11666666666666667 eigv: [0.78518621]
tau: 0.11666666666666667 eigv: [-1.17518621]
numerical: [-1.2783885 +0.j  0.80947025+0.j]

In [71]:
eigvchn_theo

array([[-0.39      , -0.39      ],
       [-0.60712255,  0.21712255],
       [-0.74424038,  0.35424038],
       [-0.85339578,  0.46339578],
       [-0.94686767,  0.55686767],
       [-1.02994012,  0.63994012],
       [-1.10546417,  0.71546417],
       [-1.17518621,  0.78518621],
       [-1.24026791,  0.85026791],
       [-1.30152835,  0.91152835]])

In [72]:
### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius

In [73]:
#### print trial by trial the first two eigenvallues
for ktrial in range(trials):
    eigvchn = eigvchn_series[ktrial,:,:].copy()
    eigvchn_real = (eigvchn)
# ### delete the 17th trial
# eigvchn_series = np.delete(eigvchn_series,17,axis=0) #for ConnStats_Adjacency_12OctDense_.npz
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

effective trials: [ 7  3  6  8 17 24  1  5 26 14 10 28 27  0 16 25 22 20 31 11 29 32 15 30
  9 34 13  2 19 33] 30


In [74]:
### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius

### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(6,4))
### label is r_{bulk}^{num}
ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

alphass= 0.25

ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',label='theoretical',alpha=alphass)
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green',alpha=alphass)

### plot error bar  
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)

ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-6.5,3.5])
ax.set_yticks([-5,0,3])
# ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
### plot y==1 
ax.axhline(y=1, color='k', linestyle='-')
plt.show()

In [75]:
# ./run_secorder 1000 0.2 0.0 0.0 0.0 0 5
# ./run_secorder 1000 0.2 0.2 0.1 0.1 1 5
# ./run_secorder 1000 0.2 0.4 0.2 0.2 1 5
# ./run_secorder 1000 0.2 0.6 0.3 0.3 1 5
# ./run_secorder 1000 0.2 0.8 0.4 0.4 1 5
# ./run_secorder 1000 0.2 1.0 0.5 0.5 1 5
# ./run_secorder 1000 0.2 1.2 0.6 0.6 1 5
# ./run_secorder 1000 0.2 1.4 0.7 0.7 1 5
# ./run_secorder 1000 0.2 1.6 0.8 0.8 1 5
# ./run_secorder 1000 0.2 1.8 0.9 0.9 1 5

In [76]:
# for seed in 1 4 8 10 13 14 18 19 25 26 31 32 34 36 40 43 44 49 52 53 55 60 62 64 71 73 76 78 81 82 83 84 85 86 87 88 93 94 95; do
# ./run_secorder 1500 0.2 0.0 0.0 0.0 0 $seed
# ./run_secorder 1500 0.2 0.2 0.1 0.1 1 $seed
# ./run_secorder 1500 0.2 0.4 0.2 0.2 1 $seed
# ./run_secorder 1500 0.2 0.6 0.3 0.3 1 $seed
# ./run_secorder 1500 0.2 0.8 0.4 0.4 1 $seed
# ./run_secorder 1500 0.2 1.0 0.5 0.5 1 $seed
# ./run_secorder 1500 0.2 1.2 0.6 0.6 1 $seed
# ./run_secorder 1500 0.2 1.4 0.7 0.7 1 $seed
# ./run_secorder 1500 0.2 1.6 0.8 0.8 1 $seed
# ./run_secorder 1500 0.2 1.8 0.9 0.9 1 $seed
# done

In [77]:
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))

thl = 1.5
ths = 1.5
                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])
            mean_leig0vec_num_series[ktrial,it,ir,0] = np.mean(eiglvec0_series[ktrial,it,:NE,ir])
            mean_leig0vec_num_series[ktrial,it,ir,1] = np.mean(eiglvec0_series[ktrial,it,NE:,ir])
            
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6#3
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        

In [79]:
mean_rvec = np.zeros((ntau,2,2))
mean_lvec = np.zeros((ntau,2,2))

for it in range(ntau):
    for ir in range(2):
        for ip in range(2):
            mean_rvec[it,ir,ip] = np.nanmean(mean_reigvec_num_series[:,it,ir,ip])
            mean_lvec[it,ir,ip] = np.nanmean(mean_leigvec_num_series[:,it,ir,ip])*eigvchn_theo[it,ir].real
            
### compute the theoretical response function 
rank = 2
response_func_contribution = np.zeros((ntau,rank,2))

### compute the theoretical response function 
response_func = np.zeros((trials,ntau,2))
for it in range(ntau):
    response_func[:,it,1] = 1
    if it>=0:#=1:
        for ir in range(2):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)       
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
    else:
        for ir in range(1):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)

In [482]:
# params = {'g':g,
#           'gamma':gamma,
#           'NE':NE,
#           'NI':NI,
#           'J':J,
#           'ce':ce,
#           'ci':ci,
#           'tau_series':tau_series,
#           }
# lst = [eigvchn_series, eigrvec_series, eiglvec_series,
#        eigrvec_series_rec, eiglvec_series_rec,
#         eiglvec0_series, norml0_series, params,
#         lowrank_eq, firing_rateeq,lowrank_eq_num,firing_ratepert,
#         # temporal_fr, temporal_perturb_fr,
#         ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,
#         contributions_lr, contributions_lr_num
#        ]
# stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
#        "eigrvec_series_rec, eiglvec_series_rec,"
#         "eiglvec0_series, norml0_series, params,"
#         "lowrank_eq, firing_rateeq,lowrank_eq_num,firing_ratepert,"
#         # "temporal_fr, temporal_perturb_fr,"
#         "ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,"
#         "contributions_lr, contributions_lr_num"]
# data = list_to_dict(lst=lst, string=stg)
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_AdjGau_sonets_31Jul_t200ms_PRX_N1500_nonneg_nonparadoxical_INHparadoxical.npz"
# np.savez(data_name, **data)
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_60N1500_normlr_whilecode_used_vecrec.npz"
### load the stored data, in particular, extracting variable: eigvchn_series 
data = np.load(data_name,allow_pickle=True)

### also loading other variables 
firing_rateeq = data['firing_rateeq']
firing_ratepert = data['firing_ratepert']
eigvchn_series = data['eigvchn_series']

eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
eiglvec0_series = data['eiglvec0_series']

In [483]:
eigvchn_series[:,-2,1]

array([0.79892975+0.j, 0.81109872+0.j, 0.78121763+0.j, 0.88953095+0.j,
       0.96582217+0.j, 0.93260863+0.j, 0.86149794+0.j, 0.85084626+0.j,
       0.85082163+0.j, 0.89128061+0.j, 0.81795933+0.j, 0.89356862+0.j,
       0.83977567+0.j, 0.84087807+0.j, 0.94135231+0.j, 0.77894884+0.j,
       0.84271445+0.j, 0.87938629+0.j, 0.87155274+0.j, 0.75986666+0.j,
       0.91819013+0.j, 0.92069713+0.j, 0.93465214+0.j, 0.85564307+0.j,
       0.84892887+0.j, 0.87203328+0.j, 0.84246614+0.j, 0.80735773+0.j,
       0.80371076+0.j, 0.84648082+0.j, 0.86119952+0.j, 0.8492122 +0.j,
       0.89259725+0.j, 0.89331787+0.j, 0.8447942 +0.j, 0.88289223+0.j])

In [484]:
#### print trial by trial the first two eigenvallues
for ktrial in range(trials):
    eigvchn = eigvchn_series[ktrial,:,:].copy()
    eigvchn_real = (eigvchn)
# ### delete the 17th trial
# eigvchn_series = np.delete(eigvchn_series,17,axis=0) #for ConnStats_Adjacency_12OctDense_.npz
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

tau_series = ((1+alphachn_series)*c**2-c**2)/(1-c)/c
### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius

### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(6,4))
# ### label is r_{bulk}^{num}
# ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
# ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
# ax.set_xlabel(r'$\tau$')
# ax.set_ylabel('Radius')
# ax.legend()
# plt.show()

alphass= 0.95

ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',label='theoretical',alpha=alphass)
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green',alpha=alphass)

### plot error bar  
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)

ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-1.5,1.])
ax.set_yticks([-1.5,0,1])
# ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
### plot y==1 
ax.axhline(y=1, color='k', linestyle='-')
plt.show()

effective trials: [ 7  3  6  8 17 24  1  5 26 14 10 28 27  0 16 25 22 20 31 11 29 32 15 30
  9 34 13  2 19 33] 30


In [486]:
# ### 
# trials = 36
# ntau = 10
# plt.close('all')
### compute the trial averaged mean 
mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leig0vec_series = np.zeros((trials,ntau,2,2))
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))
thl = 1.5
ths = 1.5
                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir]*eigvchn_series[ktrial,it,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir]*eigvchn_series[ktrial,it,ir])
            mean_leig0vec_num_series[ktrial,it,ir,0] = np.mean(eiglvec0_series[ktrial,it,:NE,ir])
            mean_leig0vec_num_series[ktrial,it,ir,1] = np.mean(eiglvec0_series[ktrial,it,NE:,ir])
            
## select the middle 30 values 
kktrial = np.arange(trials)
cuts = 5
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
    
            
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
mean_rvec = np.zeros((ntau,2,2))
mean_lvec = np.zeros((ntau,2,2))

for it in range(ntau):
    for ir in range(2):
        for ip in range(2):
            mean_rvec[it,ir,ip] = np.nanmean(mean_reigvec_num_series[:,it,ir,ip])
            mean_lvec[it,ir,ip] = np.nanmean(mean_leigvec_num_series[:,it,ir,ip])#*eigvchn_theo[it,ir]
            
### compute the theoretical response function 
rank = 2
response_func_contribution = np.zeros((ntau,rank,2))
### compute the theoretical response function 
response_func = np.zeros((trials,ntau,2))
it = 0
response_func[:,it,1] = 1
for ir in range(1):
    response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
    response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
    
    response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
    response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)

    
for it in range(1,ntau):
    response_func[:,it,1] = 1
    if eigvchn_theo[it,1]<10:#np.mean(radius_num_map[:,it]):
        # response_func[:,it,:] = np.nan
        # response_func_contribution[it,:,:] = np.nan
        # continue  
        for ir in range(2):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)  
    else:
        for ir in range(2):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
        
        

C:\Users\Administrator\AppData\Local\Temp\ipykernel_23508\3132733486.py:26: ComplexWarning: Casting complex values to real discards the imaginary part
  mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir]*eigvchn_series[ktrial,it,ir])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23508\3132733486.py:27: ComplexWarning: Casting complex values to real discards the imaginary part
  mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir]*eigvchn_series[ktrial,it,ir])


In [487]:
### 
meanfr_eq = np.zeros((trials,ntau,2))
meanfr_pert = np.zeros((trials,ntau,2))
meanfr_eq[:,:,0] = np.mean(firing_rateeq[:,:,:NE],axis=2)
meanfr_eq[:,:,1] = np.mean(firing_rateeq[:,:,NE:],axis=2)
### same for pydll Creates ()
meanfr_pert[:,:,0] = np.mean(firing_ratepert[:,:,:NE],axis=2)
meanfr_pert[:,:,1] = np.mean(firing_ratepert[:,:,NE:],axis=2)
### numerical response function 
response_func_num = np.zeros((trials,ntau,2))
for ktrial in range(trials):
    for it in range(ntau):
        response_func_num[ktrial,it,0]=(meanfr_pert[ktrial,it,0]-meanfr_eq[ktrial,it,0])/Ipert[-1]
        response_func_num[ktrial,it,1]=(meanfr_pert[ktrial,it,1]-meanfr_eq[ktrial,it,1])/Ipert[-1]
 
cuts = 10       
#### delete the largest 3 and the smallest 3 from response_func_num
for it in range(ntau):   
    idxsort = np.argsort(response_func_num[:,it,0].real)
    # print(response_func_num[idxsort,it,0])
    idxnan = idxsort[:cuts]#np.append(idxsort[:cuts],idxsort[-cuts:]) 
    response_func_num[idxnan,it,0] = np.nan
    
    idxsort = np.argsort(response_func_num[:,it,1].real)
    idxnan = idxsort[:cuts]#np.append(idxsort[:cuts],idxsort[-cuts:])
    response_func_num[idxnan,it,1] = np.nan
    
    ### find the eigenvalues > 1 
    idxeigv = np.where(np.abs(eigvchn_series[:,it,1])>1)
    idxsort = np.where(np.abs(response_func_num[:,it,0].copy())>1e1)
    idxnan = (idxsort) 
    response_func_num[idxnan,it,0] = np.nan 
    
    idxsort = np.where(np.abs(response_func_num[:,it,1].copy())>1e1)
    idxnan = (idxsort) 
    response_func_num[idxnan,it,1]  = np.nan 
    response_func_num[idxeigv,it,0] = np.nan
    response_func_num[idxeigv,it,1] = np.nan

In [489]:
### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
fige, axe = plt.subplots(figsize=(5,3))
# axe.plot(tau_series,response_func_contribution[:,0,0],'orange',marker='',alpha=0.36)
# axe.plot(tau_series,response_func_contribution[:,1,0],'green',marker='',alpha=0.36)
# axe.plot(tau_series,response_func_mean[:,0],'k',marker='')
### numerical using filled_between 
alphass = 0.95
axe.errorbar(tau_series[:]-dtau/8,response_func_num_mean[:,0],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,0],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
axe.set_xlabel(r'$\tau$')
axe.set_ylabel('response function')
axe.set_title('excitatory population')
axe.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
axe.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
axe.spines['top'].set_color('none')
axe.spines['right'].set_color('none')
axe.spines['left'].set_position(('data',tau_series[0]))
### set yaxis, ylim not too large!
axe.set_ylim([-2,2])
axe.set_yticks([-2,2])
axe.set_ylim([-2,0.5])
axe.set_yticks([-2,0.5])
axe.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
axe.set_xticks([tau_series[0],0.15,tau_series[-1]])
plt.show()

In [491]:
### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
figi, axi = plt.subplots(figsize=(5,3))
# axi.plot(tau_series,response_func_contribution[:,0,1],'orange',marker='',alpha=0.36)
# axi.plot(tau_series,response_func_contribution[:,1,1],'green',marker='',alpha=0.36)
# axi.plot(tau_series,response_func_mean[:,1],'k',marker='')
alphass = 0.95
axi.errorbar(tau_series[:]-dtau/8,response_func_num_mean[:,1],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,1],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)

# ax.fill_between(tau_series,response_func_num_mean[:,1]+np.std(response_func_num[:,:,:],axis=0)[:,1],response_func_num_mean[:,1]-np.std(response_func_num[:,:,:],axis=0)[:,1],color='black',alpha=0.3)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
axi.set_xlabel(r'$\tau$')
axi.set_ylabel('response function')
axi.set_title('inhibitory population')
axi.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
axi.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
axi.spines['top'].set_color('none')
axi.spines['right'].set_color('none')
axi.spines['left'].set_position(('data',tau_series[0]))
## set yaxis, ylim not too large!
axi.set_ylim([-1,1])
axi.set_ylim([-1,1.5])
axi.set_ylim([-1,0.5])
axi.set_ylim([-1.5,0.5])
axi.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
axi.set_xticks([tau_series[0],0.15,tau_series[-1]])
axi.set_yticks([-1.5,0.5])
plt.show()

Gaussian Approximation model

In [492]:
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/equivalentGaussian/ConnStats_WD_AdjGau_22May_t200ms_PRX_N1500_nonneg_nonparadoxical_INHparadoxical.npz"#ConnStats_WD_AdjGau_22May_t200ms_PRX_N1500_nonneg_nonparadoxical_INHparadoxical.npz"
# data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/equivalentGaussian/ConnStats_WD_AdjGau_22May_t200ms_PRX_N1500_nonneg_nonparadoxical_INHparadoxical.npz"#

# np.savez(data_name, **data)
### load data  
data = np.load(data_name,allow_pickle=True)
eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
eiglvec0_series = data['eiglvec0_series']
norml0_series = data['norml0_series']
params = data['params']
intg_ov_series = data['intg_ov_series']
first_perturb_ov = data['first_perturb_ov']
first_perturb_ovP = data['first_perturb_ovP']
intg_mean_series = data['intg_mean_series']
intg_std_series = data['intg_std_series']
intg_std_num_series = data['intg_std_num_series']

firing_rateeq = data['firing_rateeq']
firing_ratepert = data['firing_ratepert']

g = params[()]['g']
gamma = params[()]['gamma']
NE = params[()]['NE']
NI = params[()]['NI']
J = params[()]['J']
ce = params[()]['ce']
ci = params[()]['ci']
tau_series = params[()]['tau_series']

In [493]:
# ### plot a shifting histogram for the first eigenvalue
# fig,ax=plt.subplots(figsize=(12,6))
# ### plot the mean of the eigenvalues, how it changes with tau_series
# ### colormap gradiently changes with the tau_series
# cm = plt.cm.YlOrRd
# # ### the mean of the first eigenvalue
# # mean_eigvchn = np.mean(eigvchn_series[idx_eff,:,0].real,axis=0)
# # sc=ax.scatter(mean_eigvchn,-0.1*np.ones(ntau),c=tau_series,cmap=cm)
# # ### the mean of the second eigenvalue 
# # cm = plt.cm.YlGn
# # mean_eigvchn = np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)
# # sc=ax.scatter(mean_eigvchn,0.1*np.ones(ntau),c=tau_series,cmap=cm)

# # ax.set_xlabel(r'$\Re(\lambda)$',fontsize=14)
# # ### set double ylabels
# # ax.set_ylabel(r'$\Im(\lambda)$',fontsize=14)
# # ax2 = ax.twinx() 
 
# # color = 'tab:gray'
# # ax2.tick_params(axis ='y', labelcolor = color) 
# # ax2.set_yticks([])
# # ax2.spines['bottom'].set_color('none')
# # ### set other x-axis invisible
# # ax2.spines['top'].set_color('none')
# # ax2.spines['right'].set_color('none')
# # ax2.spines['left'].set_color('none')
# # # ax.set_title('Eigenvalue spectrum',fontsize=14,fontweight='bold')
# # ### refine the plot 
# # # ax.set_xlim([-3,2])
# # # ax.set_ylim([-0.3,0.3])
# # ax.set_xlim([-4,2.])
# # ax.set_ylim([-0.3,0.3])

# ax.set_aspect('equal')
# ax.spines['bottom'].set_position(('data',0))
# ### set other x-axis invisible
# ax.spines['top'].set_color('none')
# ax.spines['right'].set_color('none')
# ax.spines['left'].set_position(('data',0))
# ## set the x and y ticks
# # ax.set_xticks([-3,0,2])
# # ax.set_yticks([-0.5,0,0.5])
# # # ### also, plot the theoretical predictions for the first and second eigenvalues
# # # ax.scatter(eigvchn_theo[:,0].real,np.zeros(ntau),cmap=plt.cm.YlOrRd, c=tau_series, s=50,alpha=1,marker='^')
# # # ax.scatter(eigvchn_theo[:,1].real,np.zeros(ntau),cmap=plt.cm.YlGn, c=tau_series, s=50,alpha=1,marker='^')

# # ### plot a circle with the radius of the last radius_num_map
# # ax.add_patch(plt.Circle((0,0),radius_theo_map[-1],color='tab:blue',fill=False,linewidth=1.5,alpha=1))
# # ### plot a circle with the radius of the first radius_num_map
# # ax.add_patch(plt.Circle((0,0),radius_theo_map[0],color='tab:purple',fill=False,linewidth=1.5,alpha=0.5,linestyle='--'))

# ktrial_index = 14#16
# ### scatter plot the eigenvalue bulk of this trial
# ### randomly select 500 points
# idx_random = np.random.choice(np.arange(0,N),size=N,replace=False)
# # ax.scatter(eigvchn_series[ktrial_index,0,idx_random].real,eigvchn_series[ktrial_index,0,idx_random].imag,color='tab:purple', s=5,alpha=0.15)
# ax.scatter(eigvchn_series[ktrial_index,-1,idx_random].real,eigvchn_series[ktrial_index,-1,idx_random].imag,color='tab:blue', s=15,alpha=0.6)
# ### also plot the original eigenvalue outlier: JE-JI, using red color and X marker
# ax.scatter(JE-JI,0,color='black',marker='X',s=80,alpha=1)


In [494]:
#### print trial by trial the first two eigenvallues
for ktrial in range(trials):
    eigvchn = eigvchn_series[ktrial,:,:].copy()
    eigvchn_real = (eigvchn)
# ### delete the 17th trial
# eigvchn_series = np.delete(eigvchn_series,17,axis=0) #for ConnStats_Adjacency_12OctDense_.npz
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius

alphass= 0.35

### plot error bar  
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='^',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)

ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='^',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-1.5,1.])
ax.set_yticks([-1.5,0,1])
# ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
### plot y==1 
ax.axhline(y=1, color='k', linestyle='-')
plt.show()

effective trials: [11 33  7 24 10 21 28 35  8  9 23 34 18  4 14 15  3 13 31 20 25  2 12 30
 27 16 32  1 19  0] 30


In [495]:
# '''case 2'''
# nvec, mvec = np.zeros((N,1)), np.ones((N,1))
# nvec[:NE,0], nvec[NE:,0] = JE/NE, -JI/NI
# def case1eigv(x,J,g,NE,NI,c,tau):
#     tauc = np.abs(tau)
#     zee = J**2*c*(1-c)*NE*tauc-g*J**2*c*(1-c)*NI*0
#     zii = -g*J**2*c*(1-c)*NE*0+g**2*J**2*c*(1-c)*NI*tauc 
#     # print((NE*zee+NI*zii)*4+(JE-JI)**2)
#     z2lambda = np.zeros((N,N))
#     z2lambda[:,:NE], z2lambda[:,NE:] = zee/x**2,zii/x**2 
#     z2lambda = np.eye(N)-z2lambda 
#     final = nvec.T@la.inv(z2lambda)@mvec 
#     final = np.squeeze(final)
#     return final-x

# ### solve x 
# eigvchn_theo = np.zeros((ntau,2))
# for it, tau in enumerate(tau_series):
#     if it<=-1:
#         continue
#     x = fsolve(case1eigv,np.mean(eigvchn_series[:,it,1].real),args=(J,g,NE,NI,c,tau))
#     eigvchn_theo[it,1] = x 
#     print('tau:',tau,'eigv:',x)
#     x = fsolve(case1eigv,np.mean((eigvchn_series[:,it,0]).real),args=(J,g,NE,NI,c,tau))
#     eigvchn_theo[it,0] = x 
    
#     print('tau:',tau,'eigv:',x)
#     print('numerical:',np.mean(eigvchn_series[:,it,:2],axis=0))


In [496]:
# ### 
# trials = 36
# ntau = 10
# plt.close('all')
### compute the trial averaged mean 
mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leig0vec_series = np.zeros((trials,ntau,2,2))
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))
thl = 1.5
ths = 1.5
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,1],axis=0)
            
            mean_reigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,1],axis=0)
            
            mean_leigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,0],axis=0)
            mean_leigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,0],axis=0)
            # mean_leig0vec_series[ktrial,it,ir,0] = np.mean(leig0mean_series[ktrial,it,:NE,ir])
            # mean_leig0vec_series[ktrial,it,ir,1] = np.mean(leig0mean_series[ktrial,it,NE:,ir])
                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir]*eigvchn_series[ktrial,it,ir].real)
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir]*eigvchn_series[ktrial,it,ir].real)
            mean_leig0vec_num_series[ktrial,it,ir,0] = np.mean(eiglvec0_series[ktrial,it,:NE,ir])
            mean_leig0vec_num_series[ktrial,it,ir,1] = np.mean(eiglvec0_series[ktrial,it,NE:,ir])
            
## select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
    
            
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
        
        

In [497]:
mean_rvec = np.zeros((ntau,2,2))
mean_lvec = np.zeros((ntau,2,2))

for it in range(ntau):
    for ir in range(2):
        for ip in range(2):
            mean_rvec[it,ir,ip] = np.nanmean(mean_reigvec_num_series[:,it,ir,ip])
            mean_lvec[it,ir,ip] = np.nanmean(mean_leigvec_num_series[:,it,ir,ip])#*eigvchn_theo[it,ir]
            
### compute the theoretical response function 
rank = 2
response_func_contribution = np.zeros((ntau,rank,2))
### compute the theoretical response function 
response_func = np.zeros((trials,ntau,2))
it = 0
response_func[:,it,1] = 1
for ir in range(1):
    response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
    response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
    
    response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
    response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)

    
for it in range(1,ntau):
    response_func[:,it,1] = 1
    if eigvchn_theo[it,1]<10:#np.mean(radius_num_map[:,it]):
        # response_func[:,it,:] = np.nan
        # response_func_contribution[it,:,:] = np.nan
        # continue  
        for ir in range(2):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            # response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            # response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            
            # response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            # response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))


    
    else:
        for ir in range(2):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            # response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            # response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            
            # response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            # response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))
        

In [498]:
### 
meanfr_eq = np.zeros((trials,ntau,2))
meanfr_pert = np.zeros((trials,ntau,2))
meanfr_eq[:,:,0] = np.mean(firing_rateeq[:,:,:NE],axis=2)
meanfr_eq[:,:,1] = np.mean(firing_rateeq[:,:,NE:],axis=2)
### same for pydll Creates ()
meanfr_pert[:,:,0] = np.mean(firing_ratepert[:,:,:NE],axis=2)
meanfr_pert[:,:,1] = np.mean(firing_ratepert[:,:,NE:],axis=2)
### numerical response function 
response_func_num = np.zeros((trials,ntau,2))
for ktrial in range(trials):
    for it in range(ntau):
        response_func_num[ktrial,it,0]=(meanfr_pert[ktrial,it,0]-meanfr_eq[ktrial,it,0])/Ipert[-1]
        response_func_num[ktrial,it,1]=(meanfr_pert[ktrial,it,1]-meanfr_eq[ktrial,it,1])/Ipert[-1]
        
#### delete the largest 3 and the smallest 3 from response_func_num
for it in range(ntau):
    idxsort = np.where(np.abs(response_func_num[:,it,0].copy())>10)
    idxnan = (idxsort) 
    response_func_num[idxnan,it,0] = np.nan 
    
    
    idxsort = np.where(np.abs(response_func_num[:,it,1].copy())>10)
    idxnan = (idxsort) 
    response_func_num[idxnan,it,1] = np.nan 

In [499]:
### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
# fige, axe = plt.subplots(figsize=(5,3))
axe.plot(tau_series,response_func_contribution[:,0,0],'orange',marker='',alpha=0.36)
axe.plot(tau_series,response_func_contribution[:,1,0],'green',marker='',alpha=0.36)
axe.plot(tau_series,response_func_mean[:,0],'k',marker='')
# ### numerical using filled_between 
alphass = 0.95
axe.errorbar(tau_series[:],response_func_num_mean[:,0],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,0],fmt='^',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)

# ### set yaxis, ylim not too large!
# axe.set_ylim([-2,2])
# axe.set_yticks([-2,2])
# axe.set_ylim([-2,0.5])
# axe.set_yticks([-2,0.5])
plt.show()

In [500]:
### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
# figi, axi = plt.subplots(figsize=(5,3))
axi.plot(tau_series,response_func_contribution[:,0,1],'orange',marker='',alpha=0.36)
axi.plot(tau_series,response_func_contribution[:,1,1],'green',marker='',alpha=0.36)
axi.plot(tau_series,response_func_mean[:,1],'k')
alphass = 0.95
axi.errorbar(tau_series[:],response_func_num_mean[:,1],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,1],fmt='^',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)
plt.show()

Single backup

In [501]:
# trials = 36
# alphachns     = ['0.000','0.100','0.200','0.300','0.400','0.500','0.600','0.700','0.800','0.900']
# alphachns_rec = ['0.000','0.200','0.400','0.600','0.800','1.000','1.200','1.400','1.600','1.800']
# # alphachns_arr = np.array([0.1,0.2,0.3,0.4])
# eigvchn_series = np.zeros((trials,1,N),dtype=complex)
# # tauchns_arr = alphachns_arr*c**2/c/(1-c)
# seedss = np.array([1, 4, 8, 10, 13, 14, 18, 19, 25, 26, 31, 32, 34, 36, 40, 43, 44, 49, 52, 53, 55 ,60, 62, 64, 71, 73, 76, 78, 81, 82, 83,84, 85, 86, 87, 88, 93, 94, 95])
# seedss = np.arange(1,37)
# for ktrial in range(trials):
#     seed =seedss[ktrial]
#     z_pre = 0
#     idxalpha,alphachn=6,0.6

#     file_path = 'J:\\Allen_motif\\PRXL_data\\sonets\\N1750_FIXNUM\\seed_'+str(seed)+'\\w_1750_0.171_'+alphachns_rec[idxalpha]+'_'+alphachns[idxalpha]+'_'+alphachns[idxalpha]+'_1.000_'+str(seed)+'.dat'
#     # file_path = '/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/sonets/data/data
#     Jmat = np.zeros((N,N))
#     count =0
#     with open(file_path, 'r') as file:
#         content = file.read()
#         if content == '0':
#             print('File is empty') 
#         else:
#             for line in content.split('\n'):
#                 if line:
#                     Jmat[count,:]=([(x) for x in line.split()])
#                     count +=1
#     # ### adding excitatory and inhibitory  
#     # Zmat2 = (Jmat-c)@(Jmat-c)
#     # chainsum = 0
#     # for i in range(1000):
#     #     for j in range(1000):
#     #         if i!=j:
#     #             chainsum += Zmat2[i,j]
#     # print('chainsum,',chainsum/1000/999)           
#     Jmat[:,:NE] = Jmat[:,:NE]*je
#     Jmat[:,NE:] = Jmat[:,NE:]*(-ji)

#     ktrial, it = ktrial, 0

#     ### compute the eigenvalues and eigenvectors of the matrix
#     eigvchn, eigrvec = la.eig(Jmat)
#     ## sort the eigenValues by it's norm of the real part 
#     idxeigv = np.argsort(np.abs(eigvchn))
#     idxeigv = idxeigv[::-1]
#     eigvchn, eigrvec = eigvchn[idxeigv],eigrvec[:,idxeigv]
    
#     eigvchn_,eiglvec = la.eig(Jmat.copy().T)
#     idxeigv = np.argsort(np.abs(eigvchn_))
#     idxeigv = idxeigv[::-1]
#     eigvchn_, eiglvec = eigvchn[idxeigv],eiglvec[:,idxeigv]
    
#     ### normalization
#     reig    = np.squeeze(eigrvec[:,:].copy())
#     leig0   = np.squeeze(eiglvec[:,:].copy()) 
#     normval = np.sum(reig.copy()*leig0.copy(),axis=0)
    
#     normval = np.repeat(np.reshape(normval,(1,N)),N,axis=0)
#     leig = leig0.copy()/normval.copy()### left eigenvector normalization to make <l|r>=1
#     if np.mean(reig[:NE,0])<0:
#         reig[:,0]*=-1
#         leig[:,0]*=-1
#         leig0[:,0]*=-1
#     if np.mean(reig[:NE,1])<0: ### the second rank-1 component is negative
#         reig[:,1]*=-1
#         leig[:,1]*=-1
#         leig0[:,1]*=-1    
            
#     print('seed:',seed,'alphachn:',alphachn,'idxalpha:',idxalpha)
#     print('eigenvalues',eigvchn[:2])
    
    
#     eigvchn_series[ktrial,it,:] = eigvchn[:] 